In [1]:
!pip3 install torchtext==0.17.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
from torch.hub import load_state_dict_from_url
import re

from string import digits
import spacy
from nltk.corpus import stopwords
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score
nltk.download('stopwords')
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

#Read Data

In [ ]:
root_path = "/content/drive/MyDrive/Data Science Lab/Datasets/"

train_df = pd.read_csv(root_path + 'train.tsv', sep='\t', header=None, names=LIAR_HEADER)
valid_df = pd.read_csv(root_path + 'valid.tsv', sep='\t', header=None, names=LIAR_HEADER)
test_df = pd.read_csv(root_path + 'test.tsv', sep='\t', header=None, names=LIAR_HEADER)


#Preprocessing
Remove punctuation and stop words

#Create the vocabulary

#Create Dataset

In [ ]:
from torch.nn.utils.rnn import pad_sequence
# We do the padding here because the sentences in each batch should have the same dimension.
class MyCollate:
    # pad_idx is the index for the <pad> token
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        # Batch is a tuple of sentences and their respective labels, this can be changed depending on the input fields we are using for training
        # by changing the get item function in the dataset class.
        sentences = [x[0] for x in batch]
        labels = [x[1] for x in batch]
        paded_sentences = pad_sequence(sentences, batch_first=True, padding_value = self.pad_idx)

        return paded_sentences , torch.tensor(labels)

In [ ]:
train_dataset = LiarDataset(preprocess_liar_statements(train_df), liar_vocab)
validation_dataset = LiarDataset(preprocess_liar_statements(valid_df), liar_vocab)
test_dataset = LiarDataset(preprocess_liar_statements(test_df), liar_vocab)

train_loader = DataLoader(train_dataset, batch_size=32, collate_fn = MyCollate(pad_idx=liar_vocab["<pad>"]))
validation_loader = DataLoader(validation_dataset, batch_size=32, collate_fn = MyCollate(pad_idx=liar_vocab["<pad>"]))
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn = MyCollate(pad_idx=liar_vocab["<pad>"]))

#Create the models

In [ ]:
# Create the baseline model
class BiLSTMTextClassifierModel(nn.Module):

  def __init__(self, vocab, embedding_dim, hidden_dim, number_of_labels):
    super(BiLSTMTextClassifierModel, self).__init__()
    self.number_of_labels = number_of_labels
    self.embedding = nn.Embedding(len(vocab), embedding_dim, vocab["<pad>"])
    self.rnn = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
    self.top_layer = nn.Linear(2*hidden_dim, self.number_of_labels)
    self.softmax = F.softmax

  def forward(self, x):
    embeddings = self.embedding(x)
    rnn_output, _ = self.rnn(embeddings)
    last_hidden = rnn_output[:, -1, :]
    top_layer_output = self.top_layer(last_hidden)
    output = self.softmax(top_layer_output)
    return output


model = BiLSTMTextClassifierModel(liar_vocab, 300, 128, 6)

In [ ]:
def calculate_evaluation_metrics(correct_predictions, total_test_samples, targets_list, predicted_labels_list):
    labels_array = ["pants-fire", "false", "barely-true", "mostly-true", "true"]
    accuracy = correct_predictions / total_test_samples
    precision = precision_score(targets_list, predicted_labels_list,average='micro')
    recall = recall_score(targets_list, predicted_labels_list, average='micro')
    f1 = f1_score(targets_list, predicted_labels_list, average='micro')

    print(f'accuray: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, f1: {f1:.4f}')

In [ ]:
# Train model
# TODO refactor
# Hyper parameters
batch_size = 32
num_epochs = 3
learning_rate = 0.001

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in range(num_epochs):
  training_loss = 0
  total_training_samples = 0
  model.train()
  for batch in train_loader:
      # set model to training mode
      inputs = batch[0]
      targets = batch[1]
      output = model(inputs)
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = loss_function(outputs, targets)
      loss.backward()
      optimizer.step()
      number_of_samples = len(batch)
      training_loss+=loss.item() * number_of_samples
      total_training_samples += number_of_samples
# Evaluate on the validation set after every epoch
  model.eval()
  val_loss = 0.0
  total_validation_samples = 0
  correct_validation_predictions = 0
  predicted_labels_list = []
  targets_list = []
  with torch.no_grad():
      for inputs in validation_loader:
          # create prediction
          inputs = batch[0]
          targets = batch[1]
          outputs = model(inputs)
          # calculate the loss
          val_loss = loss_function(outputs, targets)
          number_of_samples += len(inputs)
          val_loss += val_loss.item() * number_of_samples
          total_validation_samples += number_of_samples

          # calculate f-score
          _, predicted_labels = torch.max(outputs, dim=1)
          _, true_labels = torch.max(targets, dim=1)
          correct_validation_predictions += (predicted_labels == true_labels).sum().item()
          predicted_labels_list.extend(predicted_labels.tolist())
          targets_list.extend(true_labels.tolist())

  print(f"Epoch {i+1}/{num_epochs}, Train Average Loss: {(training_loss/total_training_samples):.4f}, Val Loss: {(val_loss/total_validation_samples):.4f}")
  calculate_evaluation_metrics(correct_validation_predictions, total_validation_samples, predicted_labels_list, targets_list)


In [ ]:
  calculate_evaluation_metrics(correct_validation_predictions, total_validation_samples, predicted_labels_list, targets_list)


In [ ]:
# # Use the trained model to make predictions on the test set
# model.eval()
# with torch.no_grad():
#     total_test_samples = 0
#     correct_predictions = 0

#     predicted_labels_list = []
#     targets_list = []

#     for inputs in test_loader:
#         inputs = batch[0]
#         targets = batch[1]
#         outputs = model(inputs)

#         _, predicted_labels = torch.max(outputs, dim=1)

#         total_test_samples += len(inputs)
#         correct_predictions += (predicted_labels == targets).sum().item()

#         predicted_labels_list.extend(predicted_labels.tolist())
#         targets_list.extend(targets.tolist())

#     # Calculate evaluation metrics
#     accuracy = correct_predictions / total_test_samples
#     precision = precision_score(targets_list, predicted_labels_list)
#     recall = recall_score(targets_list, predicted_labels_list)
#     f1 = f1_score(targets_list, predicted_labels_list)

#     print(f"Accuracy: {accuracy:.4f}")
#     print(f"Precision: {precision:.4f}")
#     print(f"Recall: {recall:.4f}")
#     print(f"F1-Score: {f1:.4f}")